In [1]:
import vtk

def extract_centroids_from_glyphs(input_vtp, output_csv):
    # Load the merged glyph mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(input_vtp)
    reader.Update()
    data = reader.GetOutput()

    # Use connectivity filter to group separate glyphs (spheres)
    conn_filter = vtk.vtkConnectivityFilter()
    conn_filter.SetInputData(data)
    conn_filter.SetExtractionModeToAllRegions()
    conn_filter.ColorRegionsOn()
    conn_filter.Update()

    labeled_data = conn_filter.GetOutput()
    region_array = labeled_data.GetCellData().GetArray("RegionId")
    num_regions = conn_filter.GetNumberOfExtractedRegions()

    # Map region ID → list of point IDs
    region_point_ids = {i: set() for i in range(num_regions)}

    for cell_id in range(labeled_data.GetNumberOfCells()):
        region_id = int(region_array.GetTuple1(cell_id))
        pt_ids = vtk.vtkIdList()
        labeled_data.GetCellPoints(cell_id, pt_ids)
        for j in range(pt_ids.GetNumberOfIds()):
            pid = pt_ids.GetId(j)
            region_point_ids[region_id].add(pid)

    # Compute centroids
    points = labeled_data.GetPoints()
    centroids = []
    for pid_list in region_point_ids.values():
        coords = [points.GetPoint(pid) for pid in pid_list]
        x, y, z = map(lambda c: sum(c)/len(c), zip(*coords))
        centroids.append((x, y, z))

    # Write to CSV
    with open(output_csv, "w") as f:
        f.write("x,y,z\n")
        for x, y, z in centroids:
            f.write(f"{x},{y},{z}\n")

    print(f"✅ Extracted {len(centroids)} sphere centers to: {output_csv}")


In [2]:
extract_centroids_from_glyphs("SanAndreasFault.vtp", "SanAndreasFault.csv")
#extract_centroids_from_glyphs("FaultSurface2.vtp", "FaultSurface2.csv")
#extract_centroids_from_glyphs("FaultSurface3.vtp", "FaultSurface3.csv")

✅ Extracted 0 sphere centers to: SanAndreasFault.csv


2025-05-29 07:47:26.385 (   3.042s) [         1802371]       vtkXMLReader.cxx:306    ERR| vtkXMLPolyDataReader (0x7fde1736bf70): Error opening file SanAndreasFault.vtp
2025-05-29 07:47:26.386 (   3.042s) [         1802371]       vtkExecutive.cxx:741    ERR| vtkCompositeDataPipeline (0x600000898800): Algorithm vtkXMLPolyDataReader (0x7fde1736bf70) returned failure for request: vtkInformation (0x6000010a53e0)
  Debug: Off
  Modified Time: 150
  Reference Count: 1
  Registered Events: (none)
  Request: REQUEST_INFORMATION
  ALGORITHM_AFTER_FORWARD: 1
  FORWARD_DIRECTION: 0




In [16]:
import numpy as np
from scipy.spatial import Delaunay

import numpy as np
from scipy.spatial import Delaunay

def write_geo_from_points(csv_path, output_geo="surface_from_points.geo", mesh_size=250.0):
    # Load (x, y, z) points
    points = np.loadtxt(csv_path, delimiter=",", skiprows=1)
    xy = points[:, :2]
    z = points[:, 2]

    # 2D Delaunay triangulation in (x, y)
    tri = Delaunay(xy)

    with open(output_geo, "w") as f:
        f.write("// Gmsh .geo file generated from {}\n\n".format(csv_path))

        # Write points
        for i, (x, y, z_val) in enumerate(points):
            f.write("Point({}) = {{{}, {}, {}, {}}};\n".format(i+1, x, y, z_val, mesh_size))

        f.write("\n")

        # Write triangles as discrete surface facets
        for j, simplex in enumerate(tri.simplices):
            ids = [i+1 for i in simplex]
            f.write("Line({}) = {{{}, {}}};\n".format(3*j+1, ids[0], ids[1]))
            f.write("Line({}) = {{{}, {}}};\n".format(3*j+2, ids[1], ids[2]))
            f.write("Line({}) = {{{}, {}}};\n".format(3*j+3, ids[2], ids[0]))
            f.write("Curve Loop({}) = {{{}, {}, {}}};\n".format(j+1, 3*j+1, 3*j+2, 3*j+3))
            f.write("Ruled Surface({}) = {{{}}};\n".format(j+1, j+1))

        f.write("\n// Define Physical Surface\n")
        f.write("Physical Surface(\"Surface\") = {")
        f.write(", ".join(str(j+1) for j in range(len(tri.simplices))))
        f.write("};\n")

    print(f"✅ Wrote Gmsh .geo file to: {output_geo}")



In [20]:
write_geo_from_points("pulgas_p_values_cleaned.csv", "PulgasFault.geo")
write_geo_from_points("stanford_p_cleaned.csv", "StanfordFault.geo")
#write_geo_from_surface_points("FaultSurface2.csv", "FaultSurface2.geo", mesh_size=2.0)
#write_geo_from_surface_points("FaultSurface3.csv", "FaultSurface3.geo", mesh_size=2.0)

✅ Wrote Gmsh .geo file to: PulgasFault.geo
✅ Wrote Gmsh .geo file to: StanfordFault.geo


In [3]:
import gmsh
import math
import sys

def compute_surface_area(msh_file):
    gmsh.initialize()
    gmsh.open(msh_file)

    # Get all elements of type 2 (3-node triangles)
    element_types, element_tags, node_tags = gmsh.model.mesh.getElements()
    total_area = 0.0

    for etype, elems, nodes in zip(element_types, element_tags, node_tags):
        if etype == 2:  # Triangle
            num_elems = len(elems)
            for i in range(num_elems):
                # Each triangle has 3 nodes
                n1 = nodes[3*i]
                n2 = nodes[3*i + 1]
                n3 = nodes[3*i + 2]

                # Get coordinates of each node
                x1, y1, z1 = gmsh.model.mesh.getNode(n1)[0]
                x2, y2, z2 = gmsh.model.mesh.getNode(n2)[0]
                x3, y3, z3 = gmsh.model.mesh.getNode(n3)[0]

                # Compute vectors
                v1 = [x2 - x1, y2 - y1, z2 - z1]
                v2 = [x3 - x1, y3 - y1, z3 - z1]

                # Compute cross product
                cross = [
                    v1[1]*v2[2] - v1[2]*v2[1],
                    v1[2]*v2[0] - v1[0]*v2[2],
                    v1[0]*v2[1] - v1[1]*v2[0]
                ]

                # Compute area of the triangle
                area = 0.5 * math.sqrt(cross[0]**2 + cross[1]**2 + cross[2]**2)
                total_area += area

    gmsh.finalize()
    return total_area

In [4]:
area_stanford_fault = compute_surface_area('StanfordFault.msh') / 1E6 # Convert m2 to km2
area_pulgas_fault = compute_surface_area('PulgasFault.msh') / 1E6 # Convert m2 to km2
area_montevista_fault = compute_surface_area('MVGeometry.msh') / 1E6
area_berrocal_fault = compute_surface_area('Berrocal.msh') / 1E6

print("Area of Stanford Fault: {0}".format(area_stanford_fault)) 
print("Area of Pulgas Fault: {0}".format(area_pulgas_fault)) 
print("Area of Monte Vista Fault: {0}".format(area_montevista_fault)) 
print("Area of Berrocal Fault: {0}".format(area_berrocal_fault))

Info    : Reading 'StanfordFault.msh'...
Info    : 596 entities
Info    : 5213 nodes
Info    : 6751 elements
Info    : Done reading 'StanfordFault.msh'
Info    : Reading 'PulgasFault.msh'...
Info    : 619 entities
Info    : 3858 nodes
Info    : 4772 elements
Info    : Done reading 'PulgasFault.msh'
Info    : Reading 'MVGeometry.msh'...
Info    : 70 nodes
Info    : 154 elements
Info    : Done reading 'MVGeometry.msh'
Info    : Reading 'Berrocal.msh'...
Info    : 55 nodes
Info    : 124 elements
Info    : Done reading 'Berrocal.msh'
Area of Stanford Fault: 119.05806217570318
Area of Pulgas Fault: 73.94780468487888
Area of Monte Vista Fault: 896.2489405886654
Area of Berrocal Fault: 837.7573007418524


In [5]:
import numpy as np

# Wells and Coppersmith Scenarios:

# For Reverse Faults:
a = 4.33
b = 0.90

M_stanford_fault = a + b*np.log10(area_stanford_fault)
M_pulgas_fault = a + b*np.log10(area_pulgas_fault)
M_mv_fault = a + b*np.log10(area_montevista_fault)
M_berrocal_fault = a + b*np.log10(area_berrocal_fault)

M_stanford_pulgas = a + b*np.log10(area_stanford_fault + area_pulgas_fault)
M_mv_berrocal_fault = a + b*np.log10(area_berrocal_fault + area_montevista_fault)
M_stanford_mv_fault = a + b*np.log10(area_stanford_fault + area_montevista_fault)
M_pulgas_berrocal_fault = a + b*np.log10(area_pulgas_fault + area_berrocal_fault)
M_all = a + b*np.log10(area_pulgas_fault + area_berrocal_fault + area_stanford_fault + area_montevista_fault)

print("Magnitude for just Stanford Fault (reverse parameters): {0}".format(M_stanford_fault))
print("Magnitude for just Pulgas Fault (reverse parameters): {0}".format(M_pulgas_fault))
print("Magnitude for Stanford + Pulgas Faults (reverse parameters): {0}".format(M_stanford_pulgas))
print("Magnitude for Monte Vista + Berrocal Faults (reverse parameters): {0}".format(M_mv_berrocal_fault))
print("Magnitude for Stanford + Monte Vista Faults (reverse parameters): {0}".format(M_stanford_mv_fault))
print("Magnitude for Pulgas + Berrocal Faults (reverse parameters): {0}".format(M_pulgas_berrocal_fault))
print("Magnitude for All Faults (reverse parameters): {0}".format(M_all))

Magnitude for just Stanford Fault (reverse parameters): 6.198182928610833
Magnitude for just Pulgas Fault (reverse parameters): 6.0120327569207745
Magnitude for Stanford + Pulgas Faults (reverse parameters): 6.387013459536043
Magnitude for Monte Vista + Berrocal Faults (reverse parameters): 7.245145590696687
Magnitude for Stanford + Monte Vista Faults (reverse parameters): 7.035937643445061
Magnitude for Pulgas + Berrocal Faults (reverse parameters): 6.9938689481177185
Magnitude for All Faults (reverse parameters): 7.286395999159629


In [7]:
# Wells and Coppersmith Scenarios:

# For All Faults:
a = 4.07
b = 0.98

M_stanford_fault = a + b*np.log10(area_stanford_fault)
M_pulgas_fault = a + b*np.log10(area_pulgas_fault)
M_mv_fault = a + b*np.log10(area_montevista_fault)
M_berrocal_fault = a + b*np.log10(area_berrocal_fault)

M_stanford_pulgas = a + b*np.log10(area_stanford_fault + area_pulgas_fault)
M_mv_berrocal_fault = a + b*np.log10(area_berrocal_fault + area_montevista_fault)
M_stanford_mv_fault = a + b*np.log10(area_stanford_fault + area_montevista_fault)
M_pulgas_berrocal_fault = a + b*np.log10(area_pulgas_fault + area_berrocal_fault)
M_all = a + b*np.log10(area_pulgas_fault + area_berrocal_fault + area_stanford_fault + area_montevista_fault)

print("Magnitude for just Stanford Fault (all parameters): {0}".format(M_stanford_fault))
print("Magnitude for just Pulgas Fault (all parameters): {0}".format(M_pulgas_fault))
print("Magnitude for Stanford + Pulgas Faults (all parameters): {0}".format(M_stanford_pulgas))
print("Magnitude for Monte Vista + Berrocal Faults (all parameters): {0}".format(M_mv_berrocal_fault))
print("Magnitude for Stanford + Monte Vista Faults (all parameters): {0}".format(M_stanford_mv_fault))
print("Magnitude for Pulgas + Berrocal Faults (all parameters): {0}".format(M_pulgas_berrocal_fault))
print("Magnitude for All Faults (all parameters): {0}".format(M_all))

Magnitude for just Stanford Fault (all parameters): 6.104243633376241
Magnitude for just Pulgas Fault (all parameters): 5.901546779758177
Magnitude for Stanford + Pulgas Faults (all parameters): 6.309859100383692
Magnitude for Monte Vista + Berrocal Faults (all parameters): 7.24426964320306
Magnitude for Stanford + Monte Vista Faults (all parameters): 7.0164654339735115
Magnitude for Pulgas + Berrocal Faults (all parameters): 6.970657299061516
Magnitude for All Faults (all parameters): 7.289186754640486


In [1]:
import meshio

def convert_mesh(filename):
    
    # Read the Gmsh mesh
    mesh = meshio.read(filename)
    
    # Reconstruct a clean mesh object without cell_sets
    clean_mesh = meshio.Mesh(
        points=mesh.points,
        cells=mesh.cells,  # geometric elements
        point_data=mesh.point_data,
        cell_data=mesh.cell_data,  # still include if useful
        field_data=mesh.field_data  # optional
    )
    
    # Write to VTK
    meshio.write(filename.replace('.msh','.vtk'), clean_mesh)
    print("✅ Wrote {0} without problematic cell_sets".format(filename.replace('.msh','.vtk')))




In [3]:
convert_mesh("StanfordFault.msh")
convert_mesh("PulgasFault.msh")
convert_mesh("Berrocal.msh")
convert_mesh("MVGeometry.msh")

Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to gmsh:dim_tags.

✅ Wrote StanfordFault.vtk without problematic cell_sets



Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to gmsh:dim_tags.

✅ Wrote PulgasFault.vtk without problematic cell_sets



Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to gmsh:dim_tags.

✅ Wrote Berrocal.vtk without problematic cell_sets



Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to gmsh:dim_tags.

✅ Wrote MVGeometry.vtk without problematic cell_sets
